In [1]:
def gridID(x:Int, y:Int):Int={
  return (x/20+1+500*(500-y/20-1))
}

Intitializing Scala interpreter ...

Spark Web UI available at http://192.168.174.130:4040
SparkContext available as 'sc' (version = 2.4.4, master = local[*], app id = local-1573008183233)
SparkSession available as 'spark'


gridID: (x: Int, y: Int)Int


In [10]:
def Neighbor(x:Int):List[Int]={
    // four cornor Grid
    if (x == 1)
        List(2,501,502)
    else if (x == 500)
        List(499,999,1000)
    else if (x == 249501)
        List(249001,249002,249502)
    else if (x ==250000) 
        List(249499, 249500, 249999)
    
    // four edges
    // up
    else if ( x > 1 & x < 500)
        List(x-1, x+1, x+500, x+500-1, x+500+1)
    // down
    else if ( x > 249501 & x < 250000)
        List(x-1, x+1, x-500, x-500-1, x-500+1)
    // left
    else if ( x%500 == 1 )
        List(x+500, x+500+1, x+1, x-500, x-500+1)
    //right
    else if ( x%500 == 0 )
        List(x+500, x+500-1, x-1, x-500, x-500-1)
    
    //inside Grids
    else
        List(x-500-1, x-500, x-500+1, x-1, x+1, x+500-1, x+500, x+500+1)
}

Neighbor: (x: Int)List[Int]


In [4]:
val input = sc.textFile("Points.txt") // load Points
val grids = input.map(x => (gridID(x.split(",")(0).toInt, x.split(",")(1).toInt))) // calculate the grid belongs to point i

input: org.apache.spark.rdd.RDD[String] = Points.txt MapPartitionsRDD[5] at textFile at <console>:29
grids: org.apache.spark.rdd.RDD[Int] = MapPartitionsRDD[6] at map at <console>:30


In [5]:
val gCounts=grids.map(p=>(p,1)).reduceByKey((x,y)=>x+y) // getting a table of (grid ID, count)

gCounts: org.apache.spark.rdd.RDD[(Int, Int)] = ShuffledRDD[8] at reduceByKey at <console>:26


In [7]:
var a = gCounts.sortBy({ case (id, _) => id }, ascending = true)
a.take(20).foreach(println)

(1,31)
(2,47)
(3,38)
(4,37)
(5,43)
(6,28)
(7,29)
(8,46)
(9,49)
(10,40)
(11,55)
(12,31)
(13,40)
(14,36)
(15,47)
(16,43)
(17,45)
(18,51)
(19,46)
(20,48)


a: org.apache.spark.rdd.RDD[(Int, Int)] = MapPartitionsRDD[18] at sortBy at <console>:28


In [ ]:
val gCkey = sc.broadcast(gCounts.collectAsMap) // broadcast the gCounts to all the nodes in

In [19]:
//getting a relation of (grid, one grid neighbor J, neighbor J's count)
val gg=gCounts.map(x=>(x._1, x._1))
val gNID= gg.flatMapValues(x=>Neighbor(x))
val IDgN= gNID.map(_.swap)
val jg=IDgN.join(gCounts)

gg: org.apache.spark.rdd.RDD[(Int, Int)] = MapPartitionsRDD[33] at map at <console>:32
gNID: org.apache.spark.rdd.RDD[(Int, Int)] = MapPartitionsRDD[34] at flatMapValues at <console>:33
IDgN: org.apache.spark.rdd.RDD[(Int, Int)] = MapPartitionsRDD[35] at map at <console>:34
jg: org.apache.spark.rdd.RDD[(Int, (Int, Int))] = MapPartitionsRDD[38] at join at <console>:35


In [74]:
IDgN.toDF().show()

+------+------+
|    _1|    _2|
+------+------+
|184511|185012|
|184512|185012|
|184513|185012|
|185011|185012|
|185013|185012|
|185511|185012|
|185512|185012|
|185513|185012|
|185043|185544|
|185044|185544|
|185045|185544|
|185543|185544|
|185545|185544|
|186043|185544|
|186044|185544|
|186045|185544|
|231271|231772|
|231272|231772|
|231273|231772|
|231771|231772|
+------+------+
only showing top 20 rows



In [24]:
val gNgC=jg.values.sortByKey()
gNgC.toDF().show()

gNgC: org.apache.spark.rdd.RDD[(Int, Int)] = ShuffledRDD[50] at sortByKey at <console>:26


In [34]:
val gNMean= gNgC.combineByKey(
(v) => (v, 1), (acc: (Int, Int), v) => (acc._1 + v, acc._2 + 1), (acc1: (Int, Int), acc2: (Int, Int)) => (acc1._1 + acc2._1, acc1._2 + acc2._2) ).map{ case (key, value) => (key, value._1 / value._2.toFloat) } 
gNMean.sortByKey().toDF().show()

+-----+------+
|   _1|    _2|
+-----+------+
|19021|41.875|
|23776| 45.25|
|31037| 47.25|
|34207|40.375|
|53499|44.375|
|32676|  42.5|
|53926|  44.5|
| 4926|  41.5|
|38926|48.625|
|51620|41.625|
|54685| 43.75|
|11852| 48.75|
|49508| 41.25|
|28761|45.875|
|46781|44.375|
| 4992| 47.25|
|21780|  43.5|
|61462|42.875|
|30114|46.125|
|43975|46.625|
+-----+------+
only showing top 20 rows



gNMean: org.apache.spark.rdd.RDD[(Int, Float)] = MapPartitionsRDD[93] at map at <console>:29


In [35]:
val gCgNC=gCounts.join(gNMean)
gCgNC.sortByKey().toDF().show()

+---+---------------+
| _1|             _2|
+---+---------------+
|  1|[31, 45.333332]|
|  2|     [47, 42.2]|
|  3|     [38, 46.2]|
|  4|     [37, 45.8]|
|  5|     [43, 40.2]|
|  6|     [28, 42.0]|
|  7|     [29, 42.0]|
|  8|     [46, 42.8]|
|  9|     [49, 42.0]|
| 10|     [40, 46.2]|
| 11|     [55, 40.2]|
| 12|     [31, 45.6]|
| 13|     [40, 40.2]|
| 14|     [36, 42.6]|
| 15|     [47, 42.4]|
| 16|     [43, 43.0]|
| 17|     [45, 44.8]|
| 18|     [51, 44.0]|
| 19|     [46, 46.0]|
| 20|     [48, 42.8]|
+---+---------------+
only showing top 20 rows



gCgNC: org.apache.spark.rdd.RDD[(Int, (Int, Float))] = MapPartitionsRDD[100] at join at <console>:30


In [68]:
// the I index for reach grid
// (grid X, I index)
val Iindex = gCgNC.map(x=>(x._1, (x._2._1/x._2._2)))
Iindex.toDF().show()

+------+----------+
|    _1|        _2|
+------+----------+
|185012| 1.1084337|
|185544|  1.012987|
|231772| 0.7335244|
|182316| 0.7046632|
| 23776|0.95027626|
|199516| 1.2405063|
|170792| 0.8739496|
| 32676| 1.1529412|
|103184| 0.7734139|
| 51620| 0.9129129|
| 63852| 1.0147493|
|161980|  1.367742|
| 11852| 0.9846154|
| 49508|  0.969697|
| 81508| 1.0054644|
|175684| 0.7079646|
| 72708| 0.8372093|
|  4992| 0.8042328|
|143940| 1.1463414|
| 21780| 1.1034483|
+------+----------+
only showing top 20 rows



Iindex: org.apache.spark.rdd.RDD[(Int, Float)] = MapPartitionsRDD[206] at map at <console>:30


In [88]:
var IndexRanking = Iindex.sortBy({ case (_,index ) => index }, ascending = false)
IndexRanking.toDF().show()
val IndexRankingKey = sc.broadcast(IndexRanking.collectAsMap)

+------+---------+
|    _1|       _2|
+------+---------+
| 25822|1.8227848|
| 91782|1.7857143|
|146261|1.7809187|
|148791|1.7701149|
|175580|1.7665615|
|219452|1.7614679|
|189756|1.7585139|
|178410|     1.75|
|199412|1.7449664|
|247392|1.7380953|
|203773|1.7380192|
| 22790|1.7283951|
|128952|1.7275747|
| 94147|1.7218543|
|246110| 1.719403|
|238542|1.7160883|
|210631|1.7089783|
|210215|1.7087379|
|  3105|1.7078651|
|151593|1.7070063|
+------+---------+
only showing top 20 rows



IndexRanking: org.apache.spark.rdd.RDD[(Int, Float)] = MapPartitionsRDD[274] at sortBy at <console>:29
IndexRankingKey: org.apache.spark.broadcast.Broadcast[scala.collection.Map[Int,Float]] = Broadcast(99)


In [89]:
var Step2Result = IndexRanking.map(_.swap)
Step2Result.take(50).foreach(println)

(1.8227848,25822)
(1.7857143,91782)
(1.7809187,146261)
(1.7701149,148791)
(1.7665615,175580)
(1.7614679,219452)
(1.7585139,189756)
(1.75,178410)
(1.7449664,199412)
(1.7380953,247392)
(1.7380192,203773)
(1.7283951,22790)
(1.7275747,128952)
(1.7218543,94147)
(1.719403,246110)
(1.7160883,238542)
(1.7089783,210631)
(1.7087379,210215)
(1.7078651,3105)
(1.7070063,151593)
(1.704918,173216)
(1.6986301,37415)
(1.6976744,78086)
(1.689769,118985)
(1.689441,128633)
(1.6870748,167127)
(1.6866097,20289)
(1.6856188,64339)
(1.6855346,12942)
(1.6842105,164609)
(1.6842105,193499)
(1.681356,210268)
(1.6802508,55157)
(1.6790123,195086)
(1.6774193,56328)
(1.6755162,145657)
(1.6744186,111841)
(1.6744186,233415)
(1.6695652,82400)
(1.6688741,148510)
(1.6687117,112276)
(1.6666666,131573)
(1.6645963,202012)
(1.6645963,37964)
(1.6629213,160513)
(1.6613419,211051)
(1.6594428,249468)
(1.6594428,45627)
(1.6585366,29666)
(1.6578947,209077)


Step2Result: org.apache.spark.rdd.RDD[(Float, Int)] = MapPartitionsRDD[279] at map at <console>:28


In [119]:
var top50 = sc.parallelize(Step2Result.top(50))
var topID = top50.map(x => (x._2, x._1))
top50.toDF().show()
// val df = spark.createDataFrame(top50)
// df.withColumn("_2", "1").show

+---------+------+
|       _1|    _2|
+---------+------+
|1.8227848| 25822|
|1.7857143| 91782|
|1.7809187|146261|
|1.7701149|148791|
|1.7665615|175580|
|1.7614679|219452|
|1.7585139|189756|
|     1.75|178410|
|1.7449664|199412|
|1.7380953|247392|
|1.7380192|203773|
|1.7283951| 22790|
|1.7275747|128952|
|1.7218543| 94147|
| 1.719403|246110|
|1.7160883|238542|
|1.7089783|210631|
|1.7087379|210215|
|1.7078651|  3105|
|1.7070063|151593|
+---------+------+
only showing top 20 rows



top50: org.apache.spark.rdd.RDD[(Float, Int)] = ParallelCollectionRDD[342] at parallelize at <console>:29
topID: org.apache.spark.rdd.RDD[(Int, Float)] = MapPartitionsRDD[343] at map at <console>:30


In [123]:
var NeighborIndex = IDgN.join(topID)
NeighborIndex.toDF().show()

+------+-------------------+
|    _1|                 _2|
+------+-------------------+
|247392|[247892, 1.7380953]|
|247392|[246892, 1.7380953]|
|247392|[246893, 1.7380953]|
|247392|[247393, 1.7380953]|
|247392|[247893, 1.7380953]|
|247392|[246891, 1.7380953]|
|247392|[247391, 1.7380953]|
|247392|[247891, 1.7380953]|
|128952|[129452, 1.7275747]|
|128952|[128452, 1.7275747]|
|128952|[128953, 1.7275747]|
|128952|[128453, 1.7275747]|
|128952|[129453, 1.7275747]|
|128952|[128451, 1.7275747]|
|128952|[128951, 1.7275747]|
|128952|[129451, 1.7275747]|
| 56328| [56828, 1.6774193]|
| 56328| [55828, 1.6774193]|
| 56328| [55829, 1.6774193]|
| 56328| [56329, 1.6774193]|
+------+-------------------+
only showing top 20 rows



NeighborIndex: org.apache.spark.rdd.RDD[(Int, (Int, Float))] = MapPartitionsRDD[363] at join at <console>:30


In [128]:
var Step3Result = NeighborIndex.groupByKey()
// Step3Result.toDF().show()
Step3Result.collect().foreach(println)

(56328,CompactBuffer((56828,1.6774193), (55828,1.6774193), (55829,1.6774193), (56329,1.6774193), (56829,1.6774193), (55827,1.6774193), (56827,1.6774193), (56327,1.6774193)))
(82400,CompactBuffer((82900,1.6695652), (81900,1.6695652), (81901,1.6695652), (82901,1.6695652), (82401,1.6695652), (82399,1.6695652), (81899,1.6695652), (82899,1.6695652)))
(173216,CompactBuffer((173716,1.704918), (172716,1.704918), (173717,1.704918), (173217,1.704918), (172717,1.704918), (172715,1.704918), (173215,1.704918), (173715,1.704918)))
(128952,CompactBuffer((129452,1.7275747), (128452,1.7275747), (128953,1.7275747), (128453,1.7275747), (129453,1.7275747), (128451,1.7275747), (128951,1.7275747), (129451,1.7275747)))
(247392,CompactBuffer((247892,1.7380953), (246892,1.7380953), (246893,1.7380953), (247393,1.7380953), (247893,1.7380953), (246891,1.7380953), (247391,1.7380953), (247891,1.7380953)))
(20289,CompactBuffer((19788,1.6866097), (20788,1.6866097), (20288,1.6866097), (20789,1.6866097), (19789,1.68660

Step3Result: org.apache.spark.rdd.RDD[(Int, Iterable[(Int, Float)])] = MapPartitionsRDD[372] at groupByKey at <console>:28
